In [1]:
import pandas as pd

dataset = pd.read_parquet('./reddit_dataset.parquet').T
dataset.columns = ["title", "url", "flair", "author", "content", "subreddit"]
dataset.head()

,title,url,flair,author,content,subreddit
0,Weird indecent,https://www.reddit.com/r/creepypasta/comments/...,Text Story,ILLdoggystyleYou,"So me, my parents and my sister, live on a far...",creepypasta
1,CIA The No Sleep Experiment,https://www.reddit.com/r/creepypasta/comments/...,Text Story,313deezy,The room was silent except for the hum of mach...,creepypasta
2,The Ankle Slasher,https://www.reddit.com/r/creepypasta/comments/...,Text Story,Sealington33,"So, it started about a month or so ago. I was ...",creepypasta
3,Stick Man Game,https://www.reddit.com/r/creepypasta/comments/...,Text Story,tturnttt,Past the last house at the end of the dead-end...,creepypasta
4,This is what took my mother so this is what i ...,https://www.reddit.com/r/creepypasta/comments/...,Text Story,Big-Jicama9039,"""the yzo"" or tiwtmmstiwib is e creature that h...",creepypasta


Filter series

In [2]:
import re

def detect_series_part(text):
    try:
        pattern = r'\b(?:part\s+(?:\d+|[ivxlc]+|one|two|three|four|five|six|seven|eight|nine|ten)|' \
                  r'(first|second|third|fourth|fifth|sixth|seventh|eighth|ninth|tenth|last|final)\s+part)\b'
        match = re.search(pattern, text, re.IGNORECASE)
        return True if match else False
    except Exception as e:
        return False

dataset['is_series_part'] = dataset['title'].apply(detect_series_part)
dataset['is_series_part'].value_counts()


is_series_part
False    2084
True      159
Name: count, dtype: int64

In [3]:
subdataset = dataset[dataset['is_series_part'] == False].copy()
subdataset = subdataset.drop(columns=['is_series_part', 'url', 'flair', 'author', 'subreddit'])

Concatenate title and content

In [4]:
subdataset['title'] = dataset['title'].apply(lambda x: "Title: " + x if type(x) is str else "Title:").copy()
subdataset['content'] = dataset['content'].apply(lambda x: "Content: " + x if type(x) is str else "Content:").copy()
subdataset['text'] = subdataset['title'] + "\n" + subdataset['content']

subdataset.head()

,title,content,text
0,Title: Weird indecent,"Content: So me, my parents and my sister, live...","Title: Weird indecent \nContent: So me, my par..."
1,Title: CIA The No Sleep Experiment,Content: The room was silent except for the hu...,Title: CIA The No Sleep Experiment \nContent: ...
2,Title: The Ankle Slasher,"Content: So, it started about a month or so ag...","Title: The Ankle Slasher\nContent: So, it star..."
3,Title: Stick Man Game,Content: Past the last house at the end of the...,Title: Stick Man Game\nContent: Past the last ...
4,Title: This is what took my mother so this is ...,"Content: ""the yzo"" or tiwtmmstiwib is e creatu...",Title: This is what took my mother so this is ...


In [5]:
# from transformers import GPT2Tokenizer, GPTNeoForCausalLM
# import torch
# tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# def count_tokens(text, tokenizer):
#     tokens = tokenizer.encode(text)
#     return len(tokens)

# subdataset['num_tokens'] = subdataset['text'].apply(lambda x: count_tokens(x, tokenizer))

In [6]:
# import matplotlib.pyplot as plt
# subdataset['num_tokens'].hist(bins=100)
# plt.title('Token Count Distribution')
# plt.show()

In [9]:
text = subdataset["text"].tolist()
import pickle
with open("reddit_dataset.pkl", "wb") as f:
    pickle.dump(text, f)